In [2]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch


es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '7988304683e3', 'cluster_name': 'docker-cluster', 'cluster_uuid': '98dc57mNS1ySL5geQwco4A', 'version': {'number': '8.3.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56', 'build_date': '2022-07-23T19:30:09.227964828Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
df = pd.read_csv("books.csv", sep="~")
df.head()


,title,author,genre,discription
0,Тихий Дон,\n Миха...,"['Классика', 'Классическая проза', 'Русская кл...",\n \r\n\tМихаил Шол...
1,Анна Каренина,\n Лев ...,"['Классика', 'Классическая проза', 'Русская кл...",\n Лев Толстой. Анн...
2,Идиот,\n Федо...,"['Классика', 'Классическая проза', 'Русская кл...",\n Федор Достоевски...
3,Лолита,\n Влад...,"['Классическая проза', 'Русская классика', 'Пр...",\n \r\n\tВладимир Н...
4,Два капитана,\n Вени...,"['Классика', 'Классическая проза', 'Русская кл...",\n \r\n\tВениамин К...


In [8]:
mapp = {
    "properties": {
        "title":{"type":"text", "analyzer":"russian"},
        "author":{"type":"text", "analyzer":"russian"},
        "genre":{"type":"text", "analyzer":"russian"},
        "discription":{"type":"text", "analyzer":"russian"},
    }
}
es.indices.create(index="books", mappings=mapp)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'})

In [9]:
for i, row in df.iterrows():
    doc = {
        "title" : row["title"],
        "author" : row["author"],
        "genre" : row["genre"],
        "discription" : row["discription"]
    }
    es.index(index="books", id = i, document=doc)

In [10]:
es.indices.refresh(index="books")
es.cat.count(index="books", format="json")

ListApiResponse([{'epoch': '1670181455', 'timestamp': '19:17:35', 'count': '1918'}])

In [17]:
q1 = es.search(
    index="books",
    body= {
        "query": {
            "multi_match": {
                "query": 'заяц',
                "fields": [
                    "title",
                    "author",
                    "discription",
                ],
                "analyzer" : "russian",
            }
        }
    }
)
q1['hits']

C:\Users\tatya\AppData\Local\Temp\ipykernel_7436\2923656369.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q1 = es.search(


{'total': {'value': 8, 'relation': 'eq'},
 'max_score': 8.600743,
 'hits': [{'_index': 'books',
   '_id': '1432',
   '_score': 8.600743,
   '_source': {'title': 'Самоотверженный заяц',
    'author': '\n                                        Михаил Салтыков-Щедрин                                    ',
    'genre': "['Классика', 'Классическая проза', 'Русская классика']",
    'discription': '\n                            Михаил Салтыков-Щедрин. Самоотверженный заяц\r\n\r\n\xa0\r\n\r\nОднажды заяц перед волком провинился. Бежал он, видите ли, неподалеку от волчьего логова, а волк увидел его и кричит: «Заинька! остановись, миленький!» А заяц не только не остановился, а еще пуще ходу прибавил. Вот волк в три прыжка его поймал, да и говорит: «За то, что ты с первого моего слова не остановился, вот тебе мое решение: приговариваю я тебя к лишению живота посредством растерзания. А так как теперь и я сыт, и волчиха моя сыта,                        '}},
  {'_index': 'books',
   '_id': '1035',
  

In [18]:
q2 = es.search(
    index="books",
    body= {
        "query": {
            "multi_match": {
                "query": 'заяк',
                "fields": [
                    "title",
                    "author",
                    "discription",
                ],
                "analyzer" : "russian",
                "operator" :"and",
                "fuzziness" : 2
            }
        }
    }
)
q2['hits']

C:\Users\tatya\AppData\Local\Temp\ipykernel_7436\1648234800.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q1 = es.search(


{'total': {'value': 292, 'relation': 'eq'},
 'max_score': 9.556679,
 'hits': [{'_index': 'books',
   '_id': '1035',
   '_score': 9.556679,
   '_source': {'title': 'Здравомысленный заяц',
    'author': '\n                                        Михаил Салтыков-Щедрин                                    ',
    'genre': "['Классика', 'Классическая проза', 'Русская классика']",
    'discription': '\n                            Михаил Салтыков-Щедрин. Здравомысленный заяц\r\n\r\n\xa0\r\n\r\nХоть и обыкновенный это был заяц, а преумный. И так здраво рассуждал, что и ослу впору. Притаится под кустом, чтоб не видать его было, и сам с собой разговаривает.\r\n\r\n\xa0\r\n\r\n–\xa0Всякому, говорит, зверю свое житье предоставлено. Волку – волчье, льву – львиное, зайцу – заячье. Доволен ты или недоволен своим житьем, никто тебя не спрашивает: живи, только и всего. Нашего брата, зайца, например, все едят – кажется, имели бы                         '}},
  {'_index': 'books',
   '_id': '1571',
   '_sco

In [27]:
q3 = es.search(
  index="books",
  body= {
      "query": {
                "wildcard" : {
                  "title" : "з*я?"
                }
              },
              "fields": [
                  "title",
                  "author",
                  "discription",
              ],
          }
)
q3['hits']

C:\Users\tatya\AppData\Local\Temp\ipykernel_7436\2018630780.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q3 = es.search(


{'total': {'value': 4, 'relation': 'eq'},
 'max_score': 1.0,
 'hits': [{'_index': 'books',
   '_id': '460',
   '_score': 1.0,
   '_source': {'title': 'Заячий ремиз',
    'author': '\n                                        Николай Лесков                                    ',
    'genre': "['Классика', 'Классическая проза', 'Русская классика']",
    'discription': '\n                            Николай Лесков. Заячий ремиз\r\n\r\n\xa0\r\n\r\nНаблюдения, опыты и приключения Оноприя Перегуда из Перегудов\r\n\r\n\xa0\r\n\r\nВстань, если хотишь, на ровном месте и вели поставить вокруг себя сотню зеркал. В то время увидишь, что един твой телесный болван владеет сотнею видов, а как только зеркалы отнять, все копии сокрываются. Однако же телесный наш болван и сам есть едина токмо тень истинного человека. Сия тварь, будто обезьяна, образует лицевидным деянием невидимую и присносущную силу и божество того человека, коего в                        '},
   'fields': {'discription': ['\n             